In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier

# # import warnings filter
# from warnings import simplefilter
# # ignore all future warnings
# simplefilter(action='ignore', category=FutureWarning)

In [2]:
# dataset_path = 'allUsers.lcl.csv'
dataset_path = "../data/tommaso_data/allUsers.lcl.csv"

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)



In [3]:
from sklearn.metrics import accuracy_score, f1_score

def save_results(clf, X_test, y_test, nclf, dataf):
    dataf.loc[nclf, 'test acc'] = accuracy_score(y_test, clf.predict(X_test))
    # dataf.loc[nclf, 'test f1 score (0)'] = f1_score(y_test, clf.predict(X_test), pos_label=0)
    # dataf.loc[nclf, 'test f1 score (1)'] = f1_score(y_test, clf.predict(X_test), pos_label=1)
    dataf.loc[nclf, 'test f1 score (W)'] = f1_score(y_test, clf.predict(X_test), average='macro')
    # df.loc[nclf,'ROC AUC'] = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
    return dataf

results_df = pd.DataFrame()



# KNN

# Class recognition with User label

In [4]:
df2 = df.drop(['Class'], axis=1)
labels_Class = df['Class']


X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)


In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [6]:
print(knn.score(X_test, y_test))


0.9602432778489116


In [7]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)



# Hand owner recognition with Class label

In [8]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [9]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [10]:
print(knn.score(X_test, y_test))

0.8646606914212548


In [11]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)



# Class recognition without User label

In [12]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

In [13]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.9596670934699104


In [14]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)



# Class recognition without Class label

In [15]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)

labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [16]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.8646606914212548


In [17]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)



# SVM

In [18]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# !pip insall sklearn.grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix


# dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)

df = df.replace('?', np.nan)
df = df.dropna(thresh=len(df) * 0.8 , axis=1)
df = df.replace(np.nan, 0)

drop_indices = np.random.choice(df.index, 75000, replace=False)
df = df.drop(drop_indices)
df

,Class,User,X0,Y0,Z0,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,Z3,X4,Y4,Z4,X5,Y5,Z5
50,1,0,34.572774,81.964959,-54.302645,56.573274,72.173992,-61.975660,76.879181,42.881450,-72.217103,85.4199625095807,68.044374744442,-72.8359211633214,0,0,0,0,0,0
56,1,0,56.107673,71.549222,-63.442161,35.519543,81.381341,-54.392213,76.716773,42.096208,-72.971646,85.7467021192531,67.5559059022296,-73.2424811882883,61.069031756769,10.2621591526923,-69.5928537729302,0,0,0
90,1,0,60.180069,10.325333,-70.370529,33.186244,80.794394,-56.559278,85.466683,67.695669,-73.352786,76.0301174866085,42.3258183901331,-73.6275735151941,0,0,0,0,0,0
245,1,0,73.807648,42.206880,-75.330782,33.287076,81.310671,-56.065393,84.830206,68.114244,-73.522228,60.7460832159262,10.5260410173226,-69.5694216997701,57.4721886915503,72.1022438786633,-60.8543654826948,0,0,0
255,1,0,58.479582,72.687441,-59.007768,33.226847,81.594443,-55.290740,85.329345,68.621331,-72.757498,76.1830128922867,43.2256194892929,-72.8066952858897,59.8104891541627,11.5675391307559,-70.5414155982757,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77933,5,14,88.648724,62.148591,-59.635341,24.678448,137.390008,-29.556914,-21.343672,104.919176,-54.437631,-2.84220397831294,98.6396338519113,5.17377453974974,54.2531857723027,130.043716364429,-41.1968295308787,26.7371397994494,106.33194342428,10.5538721889144
77973,5,14,88.411788,62.176707,-59.551705,27.772889,135.767853,-33.572475,76.354893,34.884952,-39.855698,-2.38597007049348,98.7486919089091,6.20524644973614,-21.609288408928,104.669439451778,-54.6482044087237,55.4333026982379,129.769496691096,-42.2737528348901
78016,5,14,-2.315561,125.339401,-40.717868,55.353795,129.289130,-42.889666,-0.519866,97.026489,1.986116,28.7531261656966,107.691200345962,10.7625223716491,78.4080008103962,31.7976905473933,-45.7662312933219,-22.1381740228904,104.957934310698,-53.5133658265832
78049,5,14,54.971644,128.856882,-44.960160,-1.149147,122.595397,-46.846834,-29.746069,76.964799,-18.465598,-22.9503416012668,104.77633090569,-52.8738304551223,25.9709494167528,136.713223992962,-32.4013452571805,-1.50464830736565,98.0439047185886,3.7561044408024


# Class recognition without User label

In [19]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.2306451612903226


In [21]:

results_df = save_results(clf, X_test, y_test, 'SVC', results_df)


# SVC gread search class recognition without user label

In [22]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.8s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.6s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.5s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.6s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.0s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.9s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.9s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.1s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.0s
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time=   0.7s
[CV] END .....................C=1, gamma=0.1, k

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['sigmoid', 'rbf']},
             verbose=2)

In [23]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

0.8709677419354839
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}


In [24]:

results_df = save_results(grid, X_test, y_test, 'SVC', results_df)



# Hand owner recognition without Class label


In [25]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.1532258064516129


In [26]:

results_df = save_results(clf, X_test, y_test, 'SVC', results_df)



# Gread SearchHand owner recognition without Class label

In [27]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.5s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.5s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.5s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.6s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.5s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.2s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.2s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.2s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.0s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.0s
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time=   0.5s
[CV] END .....................C=1, gamma=0.1, k

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['sigmoid', 'rbf']},
             verbose=2)

In [28]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

0.682258064516129
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}


In [29]:
results_df = save_results(clf, X_test, y_test, 'SVC', results_df)



# Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [31]:
# dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)


# Class recognition with User label

In [32]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [33]:
clf = RandomForestClassifier(max_depth=200, random_state=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9863636363636363

In [34]:
results_df = save_results(clf, X_test, y_test, 'Random forest', results_df)



# Gread Search

## Show results:

In [35]:
results_df.sort_values(by=['test acc'], ascending=False)



,test acc,test f1 score (W)
Random forest,0.986364,0.940244
KNN,0.864661,0.796469
SVC,0.153226,0.055565
